In [10]:
print("Hello World!!")

Hello World!!


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


/opt/homebrew/Caskroom/miniconda/base/envs/mchat/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
PINECONE_API_KEY = "pcsk_4q9sE6_AWG6E4Ctq5sio4P3zuNnhKvTyLffdLe2qLxpbGPfZNeLr1TXJh9TjSMF5mRQx8v"
PINECONE_API_ENV = "us-east-1"

In [6]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

In [7]:
extracted_data = load_pdf("data")

In [8]:
extracted_data

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Macintosh)', 'creationdate': '2017-12-05T18:01:33+05:30', 'author': 'DK', 'moddate': '2017-12-06T18:17:07+05:30', 'title': 'Medical Symptoms, A Visual Guide', 'ebx_publisher': '/DK Publishing', 'source': 'data/Dorling Kindersley - Medical Symptoms_ A Visual Guide_ The Easy Way to Identify Medical Problems-DK Publishing (2018).pdf', 'total_pages': 258, 'page': 0, 'page_label': 'CA'}, page_content=''),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Macintosh)', 'creationdate': '2017-12-05T18:01:33+05:30', 'author': 'DK', 'moddate': '2017-12-06T18:17:07+05:30', 'title': 'Medical Symptoms, A Visual Guide', 'ebx_publisher': '/DK Publishing', 'source': 'data/Dorling Kindersley - Medical Symptoms_ A Visual Guide_ The Easy Way to Identify Medical Problems-DK Publishing (2018).pdf', 'total_pages': 258, 'page': 1, 'page_label': 'CB'}, page_content=''),
 Doc

In [9]:
# create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
chunks = text_split(extracted_data)
print("Length of my chunks: ", len(chunks))

Length of my chunks:  1529


In [11]:
chunks[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Macintosh)', 'creationdate': '2017-12-05T18:01:33+05:30', 'author': 'DK', 'moddate': '2017-12-06T18:17:07+05:30', 'title': 'Medical Symptoms, A Visual Guide', 'ebx_publisher': '/DK Publishing', 'source': 'data/Dorling Kindersley - Medical Symptoms_ A Visual Guide_ The Easy Way to Identify Medical Problems-DK Publishing (2018).pdf', 'total_pages': 258, 'page': 2, 'page_label': '2'}, page_content='MEDICAL\nSYMPTOMS\nA VISUAL GUIDE')

In [12]:
# downloading the model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

/var/folders/1z/gtn8cfh91kl0221mpmwmd3980000gn/T/ipykernel_2068/1684391885.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)